In [1]:
# Load data from sf
import pandas as pd
import numpy as np
from sklearn import linear_model
from matplotlib import pyplot as plt

In [4]:
sfdata = pd.read_csv("../data/sf.csv")

In [28]:
len(sfdata.PdDistrict.unique())

10

In [29]:
sfdata.PdDistrict.unique()

array(['MISSION', 'PARK', 'INGLESIDE', 'SOUTHERN', 'TENDERLOIN',
       'NORTHERN', 'TARAVAL', 'RICHMOND', 'CENTRAL', 'BAYVIEW'], dtype=object)

In [21]:
test = sfdata
test['CrimeCount'] = np.ones(len(sfdata))

In [42]:
test = sfdata.groupby('PdDistrict')
aggregate = test.aggregate(np.sum)

In [43]:
plt.bar(range(10), aggregate.CrimeCount)

<Container object of 10 artists>

In [44]:
plt.title("Distribution of Crime Overall Accross Police Districts in San Francisco")
plt.ylabel("Count")
plt.xlabel("PdDistrict")
plt.show()

In [12]:
# Convert date to actual date format. This might take a while!
sfdata.Date = sfdata['Date'].apply(lambda x: pd.to_datetime(x, format='%m/%d/%Y'))
sfdata.Time = sfdata['Time'].apply(lambda x: pd.to_datetime(x, format="%H:%M"))

In [95]:
def buckets(series, n):
    # Takes a series and returns a series mapping each element to a
    # one of n buckets.
    mi, ma = series.min(), series.max()
    buckets = np.linspace(mi, ma, n)
    print buckets
    def f(e):
        for i, el in enumerate(buckets):
            if e < el:
                return i
        return n - 1
            
    res = series.map(f)
    return res

def cleanColumns(data):
    # Used to rename the columns in our data grame to their appropriate names.
    # Also drops unnecessary columns.
    data['Latitude'] = data['Y']
    data['Longitude'] = data['X']
    data['Type'] = data['Category']
    
    print data.columns
    data = data.drop(['IncidntNum', 'Category', 'Descript', 'PdDistrict','Resolution','Address','X','Y', 'Location'], axis=1)
    
    return data

def createPartitions(data, n):
    # Remove outliers from the latitude/longitude issues
    # We know that the city lies between -130, -120 longitude
    # We also know that the citiy lies between 37 and 40 degrees latitude
    data = data[-120 > data['Longitude']][data['Longitude'] > (-130)]
    data = data[data['Latitude'] > 37][data['Latitude'] < 40]
    
    # Each row is an occurrance of a single crime. 
    # Keep around the original data
    data['Region'] =  n *  buckets(data['Latitude'], n) + buckets(data['Longitude'],n) + 1
    
    # Add in the types into the results.
    mapping = {key: i for i,key in enumerate(data['Type'].unique())}
    data['TypeIndex'] = data['Type'].map(mapping)

    # Now we can add the crime counts. 
    data['CrimeCount'] = np.ones(len(data))
    return data

def extractDateFeatures(data):
    # Creates a new data frame and returns it as copy with all the data that we're interested in
    # Create map from week days to integers
    DayOfWeek = {'Sunday': 1,
                 'Monday': 2,
                 'Tuesday': 3,
                 'Wednesday': 4,
                 'Thursday': 5,
                 'Friday': 6,
                 'Saturday': 7 }
    data['DoW'] = data['DayOfWeek'].map(DayOfWeek)
    data = data.drop('DayOfWeek', axis=1)
    print "Created Weeks"
    
    # We assume that the Date column is already in datetime format
    data['Month'] = data.Date.map(lambda x: x.month)
    data['DoM'] = data.Date.map(lambda x: x.day)
    data['Year'] = data.Date.map(lambda x: x.year) - data.Date.min().year
    data['ToD'] = data.Time.map(lambda x: x.minute)
    data['Time'] = data.Time.map(lambda x: x.value / 10**9) - data.Date.min().value / 10**9
    
    # We add an additional column that combines the month and the year into number of months since beginning
    data['TimeFeature'] = data.ix[:, ['Year', 'Month']].apply(lambda s: 12*s[0] + s[1], axis=1)
    
    data = data.drop('Date', axis=1)
    
    print "Created the time data features!"
    
    return data

def extractDataFeatures(data, n):
    # Given the input data read directly from the exported data 
    # (https://data.sfgov.org/Public-Safety/Map-Crime-Incidents-from-1-Jan-2003/gxxq-x39z)
    # We convert it into the format specified as follows:
    # We want the results to be a data frame with the following columns.:
    # -> Latitude (float)
    # -> Longtitude (float)
    # -> Region (specifies which region this data point belongs to)
    # -> DoW (1-7 results) (Day of the Week)
    # -> Month (1-12) (Month of the Year)
    # -> DoM (1-31) (Day of the Month)
    # -> Year (0->max(year) - min(year)) 
    # -> ToD (Time of Day, specified as number of minutes since start of day)
    # -> Time (int) : #minutes since earliest sample in the data set
    # -> Type (string) : Described the type of crime
    # -> TypeIndex (int): Index mapping uniquely each crime type to a value in [1...#crime types]
    # -> CrimeCount (int) : The number of crimes in this area
    
    # Remove unnecessary columns and rename
    cData = cleanColumns(data)
    
    # Created partitions. Note that this modifies the data by adding a REGION column.
    partitionedData = createPartitions(cData, n)
    
    # Now we convert the data to the correct dates and clean the data!
    finalData = extractDateFeatures(partitionedData)   
    
    # Return the results
    return finalData

In [234]:
def countCrime(d, region):
        '''
        Counts the crime in a given region, returning an array of size 144 with halucinated empty data 
        for non-existent crime periods.
        '''
        res = np.zeros(144)
        for i in range(144):
            try:
                # print d.ix[region, i].CrimeCount
                res[i] = d.ix[region, i].CrimeCount
            except (KeyError, AttributeError, IndexError) as e:
                pass
    
        return res

In [334]:
d = train.groupby(['Region', 'TimeFeature']).aggregate(np.sum)

In [338]:
d.ix[1]

,Time,Latitude,Longitude,TypeIndex,CrimeCount,DoW,Month,DoM,Year,ToD
TimeFeature,,,,,,,,,,
1,-42111130488360,489289.869052,-1586106.412011,86878,12956,53909,12956,206925,0,261974
2,-38753538760980,450286.467475,-1459648.287396,78169,11923,48450,23846,173738,0,237897
3,-42247629014940,490891.079164,-1591258.844254,86578,12998,52703,38994,205967,0,245571
4,-41405791420260,481108.741085,-1559547.517644,86446,12739,51549,50956,193854,0,244029
5,-40966999757160,476012.872116,-1543015.377493,84201,12604,53370,63020,199248,0,247154
6,-38766541175280,450441.836765,-1460134.859623,77700,11927,46158,71562,181629,0,228072
7,-40232438013600,467482.106381,-1515345.934369,81905,12378,49982,86646,193415,0,240240
8,-41893345372320,486780.037388,-1577902.159492,85535,12889,53499,103112,207412,0,252288
9,-41581316337360,483144.214964,-1566154.585387,85770,12793,50777,115137,199915,0,255284


In [339]:
def splitTrainTest(D, year=None):
    '''
    Given a data frame with the specified data, we split into a training set and a test set.
    The test data consists of us holding out a particular year from our training data.
    '''
    # We only want to keep some of the data
    D = D.ix[:, ['Year', 'TimeFeature', 'CrimeCount', 'Region']]
    
    # First, we're going to seperate by region.
    if year is None:
        test = D[D['Year'] == D['Year'].max()]
        train = D[D['Year'] != D['Year'].max()]
    else:
        test = D[D['Year'] == year]
        train = D[D['Year'] != year]
        
    # Now we keep only a subset of the columns we want, which is TimeFeature and CrimeCount
    train = train.drop('Year', axis=1)
    test = test.drop('Year', axis=1)

    print "Finished Creating Testing Set"
    
    # Now we groupby TimeFeature which consists of YearMonth string.
    trainD, testD = train.groupby(['Region', 'TimeFeature']).aggregate(np.sum), test.groupby([ 'Region', 'TimeFeature']).aggregate(np.sum)
    # return trainD
    trainRes = {}
    testRes = {}
    for region in range(1,D.Region.max()+1):
        # Training
        trainRes[region] = np.zeros((144,2))
        trainRes[region][:,0] = range(144)
        # print trainRes[region]
        trainRes[region][:,1] = countCrime(trainD, region)
        
        # Test 
        testRes[region] = np.zeros((144,2))
        testRes[region][:,0] = range(144) 
        testRes[region][:,1] = countCrime(testD, region)
    
    return trainRes, trainRes

In [97]:
results = extractDataFeatures(sfdata, 10)

Index([u'IncidntNum', u'Category', u'Descript', u'DayOfWeek', u'Date', u'Time', u'PdDistrict', u'Resolution', u'Address', u'X', u'Y', u'Location', u'Latitude', u'Longitude', u'Type'], dtype='object')
[ 37.70787902  37.72040589  37.73293276  37.74545963  37.7579865
  37.77051336  37.78304023  37.7955671   37.80809397  37.82062084]
[-122.51364206 -122.49709858 -122.4805551  -122.46401161 -122.44746813
 -122.43092464 -122.41438116 -122.39783767 -122.38129419 -122.3647507 ]
Created Weeks
Created the time data features!


In [340]:
def plotHistogram(results, n):
    fig, ax = plt.subplots( nrows=1, ncols=1 )
    ax.hist(results.Region, bins=range(n*n))
    plt.xlabel("San Francisco Region")
    plt.ylabel("Total Incidents of Reported Incidents")
    plt.title("Crime Incidents in San Francisco when Divided into {}x{} grid.".format(n,n))
    plt.savefig("figures/sf_n{}".format(n))
    plt.close(fig)

In [94]:
plotHistogram(results, 10)

In [239]:
train, test = splitTrainTest(results)

Finished Creating Testing Set


In [341]:
def getRMSE(x,y):
    return np.sqrt(np.sum((y - x)**2) / len(y))

def trainAndTest(train, test):
    # Given a set of training and testing data, train a linear regression model, tests it, and then 
    # computes the RMSE of each region, returning the resulting dictionary of RMSEs for region, as well as
    # a dictionary of predictions, and a dictionary of trained models
    models = {}
    RMSE = {}
    predictions = {}
    for region in train:
        if region % 10 == 0:
            print "Training on region {}".format(region)
        model = LinearRegression()
        x = train[region][:,0]
        x = x.reshape((len(x),1))
        y = train[region][:,1]
        y = y.reshape((len(y),1))
        model.fit(x,y)
        xTest = test[region][:,0]
        xTest = xTest.reshape((len(xTest),1))
        yTest = test[region][:,1]
        yTest = yTest.reshape((len(yTest),1))
        preds = model.predict(xTest)
        rmse = getRMSE(yTest, preds)
        
        # store the results
        models[region] = model
        RMSE[region] = rmse
        predictions[region] = preds
    
    return models, RMSE, predictions

In [292]:
models, RMSE, predictions = trainAndTest(train, test)

Training on region 0
Training on region 10
Training on region 20
Training on region 30
Training on region 40
Training on region 50
Training on region 60
Training on region 70
Training on region 80
Training on region 90


In [295]:
sum(RMSE.values()) / len(RMSE)

18.802226607258753

In [324]:
tRes = createPartitions(results, 1)

[ 37.70787902]
[-122.51364206]


In [329]:
tRes

,Time,Latitude,Longitude,Type,Region,TypeIndex,CrimeCount,DoW,Month,DoM,Year,ToD,TimeFeature
0,-3250353600,37.760888,-122.435003,ASSAULT,1,0,1,4,4,20,2,0,28
1,-3250303200,37.762255,-122.446838,LARCENY/THEFT,1,1,1,1,1,13,5,0,61
2,-3250353000,37.724931,-122.444707,ASSAULT,1,0,1,1,5,5,10,10,125
3,-3250364400,37.783288,-122.408954,DRIVING UNDER THE INFLUENCE,1,2,1,3,7,8,0,0,7
4,-3250292820,37.782793,-122.414056,OTHER OFFENSES,1,3,1,6,10,4,10,53,130
5,-3250342800,37.803467,-122.426731,BURGLARY,1,4,1,3,8,14,4,0,56
6,-3250316220,37.723129,-122.435977,DRUG/NARCOTIC,1,5,1,3,3,4,5,23,63
7,-3250311000,37.720158,-122.447241,OTHER OFFENSES,1,3,1,4,7,5,3,50,43
8,-3250333800,37.737157,-122.432788,LARCENY/THEFT,1,1,1,4,12,10,0,30,12
9,-3250311900,37.715900,-122.408761,NON-CRIMINAL,1,6,1,2,1,17,8,35,97


In [328]:
test

{0: array([[   0.,    0.],
        [   1.,    0.],
        [   2.,    0.],
        [   3.,    0.],
        [   4.,    0.],
        [   5.,    0.],
        [   6.,    0.],
        [   7.,    0.],
        [   8.,    0.],
        [   9.,    0.],
        [  10.,    0.],
        [  11.,    0.],
        [  12.,    0.],
        [  13.,    0.],
        [  14.,    0.],
        [  15.,    0.],
        [  16.,    0.],
        [  17.,    0.],
        [  18.,    0.],
        [  19.,    0.],
        [  20.,    0.],
        [  21.,    0.],
        [  22.,    0.],
        [  23.,    0.],
        [  24.,    0.],
        [  25.,    0.],
        [  26.,    0.],
        [  27.,    0.],
        [  28.,    0.],
        [  29.,    0.],
        [  30.,    0.],
        [  31.,    0.],
        [  32.,    0.],
        [  33.,    0.],
        [  34.,    0.],
        [  35.,    0.],
        [  36.,    0.],
        [  37.,    0.],
        [  38.,    0.],
        [  39.,    0.],
        [  40.,    0.],
        [  41

In [343]:
# We try different values of n and calculate the average RMSE for each value!
# Note that results already has the data extracted
rmses = []
max_rmses = []
min_rmses = []
for n in range(1,10) + range(10,50,10):
    # We only need to extract the data once, which results has already done (for n = 10)
    tRes = createPartitions(results, n)
    # This saves a plot of the distribution as a histogram
    plotHistogram(tRes, n)
    
    # Now we split into train and test
    train, test = splitTrainTest(tRes)
    
    # Now we caculate the average RMSE
    _, RMSE, _ = trainAndTest(train, test)
    print "RMSE: {}".format(sum(RMSE.values()) / len(RMSE))
    rmses.append(sum(RMSE.values()) / len(RMSE))
    max_rmses.append(max(RMSE.values()))
    min_rmses.append(min(RMSE.values()))
    

[ 37.70787902]
[-122.51364206]
Finished Creating Testing Set
1254.35693616
[ 37.70787902  37.82062084]
[-122.51364206 -122.3647507 ]
Finished Creating Testing Set
313.58923404
[ 37.70787902  37.76424993  37.82062084]
[-122.51364206 -122.43919638 -122.3647507 ]
Finished Creating Testing Set
148.256074643
[ 37.70787902  37.74545963  37.78304023  37.82062084]
[-122.51364206 -122.46401161 -122.41438116 -122.3647507 ]
Finished Creating Testing Set
Training on region 10
87.854704454
[ 37.70787902  37.73606448  37.76424993  37.79243538  37.82062084]
[-122.51364206 -122.47641922 -122.43919638 -122.40197354 -122.3647507 ]
Finished Creating Testing Set
Training on region 10
Training on region 20
59.1517495269
[ 37.70787902  37.73042739  37.75297575  37.77552411  37.79807247
  37.82062084]
[-122.51364206 -122.48386379 -122.45408552 -122.42430725 -122.39452898
 -122.3647507 ]
Finished Creating Testing Set
Training on region 10
Training on region 20
Training on region 30
44.0018879707
[ 37.70787902

In [345]:
min_rmses

[1254.3569361587199,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [346]:
x =range(1,10) + range(10,50,10)
plt.scatter(x, rmses, color='red')
plt.scatter(x, max_rmses, color='blue')
plt.title("RMSE vs Resolution")
plt.xlabel("Geographic Resolution (n)")
plt.ylabel("Average and Max RMSE")
plt.show()

In [347]:
zip(x,rmses)

[(1, 1254.3569361587199),
 (2, 313.58923403967998),
 (3, 148.25607464296834),
 (4, 87.854704454013273),
 (5, 59.151749526940783),
 (6, 44.001887970683832),
 (7, 33.574973860507114),
 (8, 27.383689024622392),
 (9, 22.730095429324372),
 (10, 18.889649014172022),
 (20, 7.0566567945401486),
 (30, 4.0394574173716151),
 (40, 2.7812893206180189)]